In [1]:
import tensorflow as tf
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import pathlib
from PIL import Image
import numpy
import os
import sys
from keras import layers



In [2]:
#set working directory
os.chdir(r'C:\Users\ethan\Desktop\mandarin_recognition_code2')

In [ ]:
train_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Train\Train')
test_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Test\Test')

'\ntrain_chinese = os.listdir(train_dir)\ntest_chinese = os.listdir(test_dir)\nlen(train_chinese)\nlen(test_chinese)\n'

In [ ]:
#make tensorflow dataset from image directory (if not already saved on local disk)


train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    seed=42,
    validation_split = 0.2,
    subset = 'both',
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)



Found 3223042 files belonging to 7330 classes.
Using 2578434 files for training.
Using 644608 files for validation.
Found 776523 files belonging to 7330 classes.


In [ ]:
#if it is first time generating dataset, save it to local disk
tf.data.Dataset.save(train_dataset, 'train_dataset')
tf.data.Dataset.save(val_dataset, 'val_dataset')
tf.data.Dataset.save(test_dataset, 'test_dataset')

In [3]:
#load dataset from local disk
train_dataset = tf.data.Dataset.load('train_dataset')
test_dataset = tf.data.Dataset.load('test_dataset')
val_dataset = tf.data.Dataset.load('val_dataset')

In [4]:
#Check shape of dataset
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (32, 56, 56, 1)
Labels shape: (32, 7330)


In [5]:
print("starting rescaling")

rescaling_layer = layers.Rescaling(scale=1./255)
train_dataset = train_dataset.map(lambda x, y: (rescaling_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (rescaling_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (rescaling_layer(x), y))

starting rescaling


In [7]:
normalization_layer = layers.Normalization(axis=None)
normalization_layer.adapt(train_dataset.map(lambda x, y: x))

In [9]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


model = tf.keras.Sequential()
model.add(layers.Input(shape=(56, 56, 1)))
model.add(normalization_layer)
model.add(layers.Conv2D(filters=80, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=160, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=320, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters= 640, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dense(units=200, activation='relu'))
model.add(layers.Dense(units=7330, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  multiple                 3         
 ion)                                                            
                                                                 
 conv2d_1 (Conv2D)           (None, 54, 54, 80)        800       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 80)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 160)       115360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 160)      0         
 2D)                                                             
                                                      

In [10]:
model.save('mandarin_classification_v1')

INFO:tensorflow:Assets written to: mandarin_classification_v1\assets


INFO:tensorflow:Assets written to: mandarin_classification_v1\assets


In [ ]:
model = tf.keras.models.load_model('mandarin_classification_v1')
model.summary()

In [ ]:
predict_img = tf.keras.utils.load_img(r"C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Test\Test\中\1.png", target_size=(56,56))
img_array = tf.keras.utils.img_to_array(predict_img)
img_array = tf.expand_dims(img_array, 0)
img_array = img_array/255

NameError: name 'tf' is not defined

In [ ]:
# Convert the image to grayscale
img_array_gray = tf.image.rgb_to_grayscale(img_array)

# Normalize the grayscale image
img_array_gray = img_array_gray / 255.0

# Predict the class
prediction = model.predict(img_array_gray)
predicted_class = numpy.argmax(prediction, axis=-1)

print(predicted_class)

1/1 [==============================] - 0s 442ms/step
[58]


In [ ]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in train_dataset:   # use dataset.unbatch() with repeat
    # append true labels
    y_true.append(label_batch)
    # compute predictions
    label_batch = tf.image.rgb_to_grayscale(image_batch)
    label_batch = tf.reshape(label_batch, (label_batch.shape[0], -1))
    label_batch = map(label_batch, layers.Rescaling(.1/255))
    preds = model.predict(image_batch)
    # append predicted labels
    y_pred.append(numpy.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [ ]:
class_names = train_dataset.element_spec
print(class_names)

(TensorSpec(shape=(None, 56, 56, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7330), dtype=tf.float32, name=None))
